In [64]:
import pandas as pd
import requests 
import json
import ast
import time
from requests.exceptions import ConnectionError, HTTPError, Timeout
from sklearn.preprocessing import StandardScaler
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
data_1 = pd.read_csv("tmdb_5000_credits.csv")
data_1.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [3]:
data_2  = pd.read_csv("tmdb_5000_movies.csv")
data_2.head()


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
def extract_cast(cast_no):
    try:
        cast = ast.literal_eval(cast_no)
        cast_names = [g['name'] for g in cast]
        return cast_names
    except:
        return []
    
data_1['cast_names'] = data_1['cast'].apply(extract_cast)

In [ ]:
def extract_crew(crew_no):
    try:
        crew = ast.literal_eval(crew_no)
        crew_names = [g['name'] for g in crew]
        return crew_names
    except:
        return []
    
data_1['crew_names'] = data_1['crew'].apply(extract_crew) 



,movie_id,title,cast,crew,cast_names,crew_names
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter..."


In [9]:
data_1.drop(columns=["cast", "crew"], inplace=True)
data_1.head()


,movie_id,title,cast_names,crew_names
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."
2,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John..."
3,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan..."
4,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter..."


In [10]:
data_1.head(10)


,movie_id,title,cast_names,crew_names
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."
2,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John..."
3,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan..."
4,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter..."
5,559,Spider-Man 3,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[Francine Maisler, Joseph M. Caracciolo Jr., S..."
6,38757,Tangled,"[Zachary Levi, Mandy Moore, Donna Murphy, Ron ...","[John Lasseter, Jacob Grimm, Wilhelm Grimm, Jo..."
7,99861,Avengers: Age of Ultron,"[Robert Downey Jr., Chris Hemsworth, Mark Ruff...","[Danny Elfman, Christopher Boyes, Christopher ..."
8,767,Harry Potter and the Half-Blood Prince,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[Bruno Delbonnel, Stephanie McMillan, Stuart C..."
9,209112,Batman v Superman: Dawn of Justice,"[Ben Affleck, Henry Cavill, Gal Gadot, Amy Ada...","[Hans Zimmer, Charles Roven, Christopher Nolan..."


In [11]:
def extract_keyword(keyword_no):
    try:
        keyword = ast.literal_eval(keyword_no)
        keyword_name = [i['name'] for i in keyword]
        return keyword_name
    except:
        return []

data_2['keyword_used'] = data_2["keywords"].apply(extract_keyword)


In [12]:
def extract_genre(genre_no):
    try:
        genre = ast.literal_eval(genre_no)
        genre_name = [f['name'] for f in genre]
        return genre_name
    except:
        return []
    
data_2['genre_movies'] = data_2['genres'].apply(extract_genre)


In [13]:
data_2.rename(columns= {"id":"movie_id"}, inplace=True)
data_2.head()

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,keyword_used,genre_movies
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]"
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]"
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]"
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]"


In [14]:
movies = data_2[["movie_id", "original_title", "overview", "keyword_used", "genre_movies", "vote_average", "vote_count", "title"]]
movies.head()

,movie_id,original_title,overview,keyword_used,genre_movies,vote_average,vote_count,title
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,Avatar
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]",6.9,4500,Pirates of the Caribbean: At World's End
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]",6.3,4466,Spectre
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]",7.6,9106,The Dark Knight Rises
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]",6.1,2124,John Carter


In [15]:
combined_data = data_1.merge(movies, on=["movie_id", "title"], how="inner")

combined_data.head(10)

,movie_id,title,cast_names,crew_names,original_title,overview,keyword_used,genre_movies,vote_average,vote_count
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]",7.2,11800
1,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]",6.9,4500
2,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John...",Spectre,A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]",6.3,4466
3,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]",7.6,9106
4,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter...",John Carter,"John Carter is a war-weary, former military ca...","[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]",6.1,2124
5,559,Spider-Man 3,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[Francine Maisler, Joseph M. Caracciolo Jr., S...",Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,"[dual identity, amnesia, sandstorm, love of on...","[Fantasy, Action, Adventure]",5.9,3576
6,38757,Tangled,"[Zachary Levi, Mandy Moore, Donna Murphy, Ron ...","[John Lasseter, Jacob Grimm, Wilhelm Grimm, Jo...",Tangled,When the kingdom's most wanted-and most charmi...,"[hostage, magic, horse, fairy tale, musical, p...","[Animation, Family]",7.4,3330
7,99861,Avengers: Age of Ultron,"[Robert Downey Jr., Chris Hemsworth, Mark Ruff...","[Danny Elfman, Christopher Boyes, Christopher ...",Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,"[marvel comic, sequel, superhero, based on com...","[Action, Adventure, Science Fiction]",7.3,6767
8,767,Harry Potter and the Half-Blood Prince,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[Bruno Delbonnel, Stephanie McMillan, Stuart C...",Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...","[witch, magic, broom, school of witchcraft, wi...","[Adventure, Fantasy, Family]",7.4,5293
9,209112,Batman v Superman: Dawn of Justice,"[Ben Affleck, Henry Cavill, Gal Gadot, Amy Ada...","[Hans Zimmer, Charles Roven, Christopher Nolan...",Batman v Superman: Dawn of Justice,Fearing the actions of a god-like Super Hero l...,"[dc comics, vigilante, superhero, based on com...","[Action, Adventure, Fantasy]",5.7,7004


In [16]:
combined_data.drop(columns=["original_title"], inplace=True)

In [17]:
combined_data.head()

,movie_id,title,cast_names,crew_names,overview,keyword_used,genre_movies,vote_average,vote_count
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B...","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]",7.2,11800
1,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe...","Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]",6.9,4500
2,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John...",A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]",6.3,4466
3,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan...",Following the death of District Attorney Harve...,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]",7.6,9106
4,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter...","John Carter is a war-weary, former military ca...","[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]",6.1,2124


In [18]:
combined_data.shape

(4803, 9)

In [ ]:
# Api Requests Format

url= "https://api.themoviedb.org/3/discover/movie"
api_key = "api_key"
all_movies = []

try:
    for page in range(1, 11):
        param = {"api_key": api_key, "language": "en-US", "page": page, "sort_by": "popularity.desc"}
        response = requests.get(url, params=param)
        movies = response.json().get("results", [])

        for movie in movies:
            movie_id = movie["id"]
        # Get full details
            detail_url = f"https://api.themoviedb.org/3/movie/{movie_id}"
            detail_params = {"api_key": api_key, "append_to_response": "credits,keywords"}
            detail_data = requests.get(detail_url, params=detail_params).json()

            movie_data = {
            "id": detail_data.get("id"),
            "title": detail_data.get("title"),
            "genres": json.dumps(detail_data.get("genres", [])),
            "overview": detail_data.get("overview"),
            "keywords": json.dumps(detail_data.get("keywords", {}).get("keywords", [])),
            "cast": json.dumps(detail_data.get("credits", {}).get("cast", [])),
            "crew": json.dumps(detail_data.get("credits", {}).get("crew", [])),
            "vote_average": detail_data.get("vote_average"),
            "vote_count": detail_data.get("vote_count")
            }
            all_movies.append(movie_data)
        print(f"Page {page} done")
        time.sleep(0.2)  # avoid rate limiting
except ConnectionError as conn_err:
    print(f"Connection Error : {conn_err}")
except HTTPError as ht_err:
    print(f"HTTPError : {ht_err}")
except Timeout as ti_o:
    print(f"Time Out : {ti_o}")


df = pd.DataFrame(all_movies)

data = df.to_csv("api_data.csv", index=False)

Page 1 done
Page 2 done
Page 3 done
Page 4 done
Page 5 done
Page 6 done
Page 7 done
Page 8 done
Page 9 done
Page 10 done


In [ ]:
# Api Requests Format

url= "https://api.themoviedb.org/3/discover/movie"
api_key = "api_key"
all_movies_end = []

try:
    for page in range(1, 501):
        param = {"api_key": api_key, "language": "en-US", "page": page, "sort_by": "popularity.desc"}
        response = requests.get(url, params=param)
        movies = response.json().get("results", [])

        for movie in movies:
            movie_id = movie["id"]
        # Get full details
            detail_url = f"https://api.themoviedb.org/3/movie/{movie_id}"
            detail_params = {"api_key": api_key, "append_to_response": "credits,keywords"}
            detail_data = requests.get(detail_url, params=detail_params).json()

            movie_data = {
            "movie_id": detail_data.get("id"),
            "title": detail_data.get("title"),
            "genres": json.dumps(detail_data.get("genres", [])),
            "overview": detail_data.get("overview"),
            "keywords": json.dumps(detail_data.get("keywords", {}).get("keywords", [])),
            "cast": json.dumps(detail_data.get("credits", {}).get("cast", [])),
            "crew": json.dumps(detail_data.get("credits", {}).get("crew", [])),
            "vote_average": detail_data.get("vote_average"),
            "vote_count": detail_data.get("vote_count")
            }
            all_movies_end.append(movie_data)
        print(f"Page {page} done")
        time.sleep(0.2)  # avoid rate limiting
except ConnectionError as conn_err:
    print(f"Connection Error : {conn_err}")
except HTTPError as ht_err:
    print(f"HTTPError : {ht_err}")
except Timeout as ti_o:
    print(f"Time Out : {ti_o}")




Page 1 done
Page 2 done
Page 3 done
Connection Error : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [21]:
add_data = pd.read_csv("api_data.csv")

add_data.head()

,id,title,genres,overview,keywords,cast,crew,vote_average,vote_count
0,1062722,Frankenstein,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 27, ""name...","Dr. Victor Frankenstein, a brilliant but egoti...","[{""id"": 1299, ""name"": ""monster""}, {""id"": 818, ...","[{""adult"": false, ""gender"": 2, ""id"": 25072, ""k...","[{""adult"": false, ""gender"": 2, ""id"": 44138, ""k...",7.873,1250
1,1248226,Playdate,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",When out-of-work accountant Brian joins stay-a...,"[{""id"": 5501, ""name"": ""stay-at-home dad""}, {""i...","[{""adult"": false, ""gender"": 2, ""id"": 32895, ""k...","[{""adult"": false, ""gender"": 2, ""id"": 2808129, ...",6.600,127
2,1242898,Predator: Badlands,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...","Cast out from his clan, a young Predator finds...","[{""id"": 10349, ""name"": ""survival""}, {""id"": 102...","[{""adult"": false, ""gender"": 1, ""id"": 18050, ""k...","[{""adult"": false, ""gender"": 2, ""id"": 1094, ""kn...",7.292,300
3,1054867,One Battle After Another,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",Washed-up revolutionary Bob exists in a state ...,"[{""id"": 387, ""name"": ""california""}, {""id"": 818...","[{""adult"": false, ""gender"": 2, ""id"": 6193, ""kn...","[{""adult"": false, ""gender"": 2, ""id"": 1102140, ...",7.660,1282
4,1197137,Black Phone 2,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...","Four years after escaping The Grabber, Finney ...","[{""id"": 2546, ""name"": ""mask""}, {""id"": 516, ""na...","[{""adult"": false, ""gender"": 2, ""id"": 569, ""kno...","[{""adult"": false, ""gender"": 2, ""id"": 84348, ""k...",7.131,467


In [22]:
add_data['keyword_used'] = add_data["keywords"].apply(extract_keyword)

add_data["genre_movies"] = add_data['genres'].apply(extract_genre)

In [23]:
def extract_casted(cast_str):
    try:
        # Convert JSON string back to Python list
        cast_list = json.loads(cast_str)
        return [i['name'] for i in cast_list]
    except:
        return []

def extract_director(crew_str):
    try:
        crew_list = json.loads(crew_str)
        return [i['name'] for i in crew_list]
    except:
        return []
    
add_data["crew_names"] = add_data["crew"].apply(extract_casted)
add_data["cast_names"] = add_data["cast"].apply(extract_director)


In [24]:
add_data.rename(columns={"id":"movie_id"}, inplace=True)
add_data.head()

,movie_id,title,genres,overview,keywords,cast,crew,vote_average,vote_count,keyword_used,genre_movies,crew_names,cast_names
0,1062722,Frankenstein,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 27, ""name...","Dr. Victor Frankenstein, a brilliant but egoti...","[{""id"": 1299, ""name"": ""monster""}, {""id"": 818, ...","[{""adult"": false, ""gender"": 2, ""id"": 25072, ""k...","[{""adult"": false, ""gender"": 2, ""id"": 44138, ""k...",7.873,1250,"[monster, based on novel or book, supernatural...","[Drama, Horror, Fantasy]","[J. Miles Dale, Keith Thompson, Dennis Berardi...","[Oscar Isaac, Jacob Elordi, Christoph Waltz, M..."
1,1248226,Playdate,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",When out-of-work accountant Brian joins stay-a...,"[{""id"": 5501, ""name"": ""stay-at-home dad""}, {""i...","[{""adult"": false, ""gender"": 2, ""id"": 32895, ""k...","[{""adult"": false, ""gender"": 2, ""id"": 2808129, ...",6.600,127,"[stay-at-home dad, accountant, action comedy]","[Action, Comedy, Family]","[Jason Benoit, Jeffrey Greenstein, Neil Goldma...","[Kevin James, Alan Ritchson, Sarah Chalke, Isl..."
2,1242898,Predator: Badlands,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...","Cast out from his clan, a young Predator finds...","[{""id"": 10349, ""name"": ""survival""}, {""id"": 102...","[{""adult"": false, ""gender"": 1, ""id"": 18050, ""k...","[{""adult"": false, ""gender"": 2, ""id"": 1094, ""kn...",7.292,300,"[survival, outcast, alien planet, grand, team ...","[Action, Science Fiction, Adventure]","[John Thomas, Jim Thomas, Patrick Aison, Dan T...","[Elle Fanning, Dimitrius Schuster-Koloamatangi..."
3,1054867,One Battle After Another,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",Washed-up revolutionary Bob exists in a state ...,"[{""id"": 387, ""name"": ""california""}, {""id"": 818...","[{""adult"": false, ""gender"": 2, ""id"": 6193, ""kn...","[{""adult"": false, ""gender"": 2, ""id"": 1102140, ...",7.660,1282,"[california, based on novel or book, usa–mexic...","[Action, Thriller, Crime]","[Adam Somner, Paul Thomas Anderson, Cassandra ...","[Leonardo DiCaprio, Sean Penn, Chase Infiniti,..."
4,1197137,Black Phone 2,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...","Four years after escaping The Grabber, Finney ...","[{""id"": 2546, ""name"": ""mask""}, {""id"": 516, ""na...","[{""adult"": false, ""gender"": 2, ""id"": 569, ""kno...","[{""adult"": false, ""gender"": 2, ""id"": 84348, ""k...",7.131,467,"[mask, child abuse, dreams, sibling relationsh...","[Horror, Thriller]","[Jason Blum, C. Robert Cargill, Scott Derricks...","[Ethan Hawke, Mason Thames, Madeleine McGraw, ..."


In [25]:
final_add_data = add_data[["movie_id", "title", "genre_movies", "keyword_used", "cast_names", "crew_names", "overview", "vote_average", "vote_count"]]
final_add_data.head()

,movie_id,title,genre_movies,keyword_used,cast_names,crew_names,overview,vote_average,vote_count
0,1062722,Frankenstein,"[Drama, Horror, Fantasy]","[monster, based on novel or book, supernatural...","[Oscar Isaac, Jacob Elordi, Christoph Waltz, M...","[J. Miles Dale, Keith Thompson, Dennis Berardi...","Dr. Victor Frankenstein, a brilliant but egoti...",7.873,1250
1,1248226,Playdate,"[Action, Comedy, Family]","[stay-at-home dad, accountant, action comedy]","[Kevin James, Alan Ritchson, Sarah Chalke, Isl...","[Jason Benoit, Jeffrey Greenstein, Neil Goldma...",When out-of-work accountant Brian joins stay-a...,6.600,127
2,1242898,Predator: Badlands,"[Action, Science Fiction, Adventure]","[survival, outcast, alien planet, grand, team ...","[Elle Fanning, Dimitrius Schuster-Koloamatangi...","[John Thomas, Jim Thomas, Patrick Aison, Dan T...","Cast out from his clan, a young Predator finds...",7.292,300
3,1054867,One Battle After Another,"[Action, Thriller, Crime]","[california, based on novel or book, usa–mexic...","[Leonardo DiCaprio, Sean Penn, Chase Infiniti,...","[Adam Somner, Paul Thomas Anderson, Cassandra ...",Washed-up revolutionary Bob exists in a state ...,7.660,1282
4,1197137,Black Phone 2,"[Horror, Thriller]","[mask, child abuse, dreams, sibling relationsh...","[Ethan Hawke, Mason Thames, Madeleine McGraw, ...","[Jason Blum, C. Robert Cargill, Scott Derricks...","Four years after escaping The Grabber, Finney ...",7.131,467


In [66]:
final_add_data.shape

(200, 9)

In [26]:
total_end = pd.concat([final_add_data, combined_data], ignore_index=True)

total_end.head()

,movie_id,title,genre_movies,keyword_used,cast_names,crew_names,overview,vote_average,vote_count
0,1062722,Frankenstein,"[Drama, Horror, Fantasy]","[monster, based on novel or book, supernatural...","[Oscar Isaac, Jacob Elordi, Christoph Waltz, M...","[J. Miles Dale, Keith Thompson, Dennis Berardi...","Dr. Victor Frankenstein, a brilliant but egoti...",7.873,1250
1,1248226,Playdate,"[Action, Comedy, Family]","[stay-at-home dad, accountant, action comedy]","[Kevin James, Alan Ritchson, Sarah Chalke, Isl...","[Jason Benoit, Jeffrey Greenstein, Neil Goldma...",When out-of-work accountant Brian joins stay-a...,6.600,127
2,1242898,Predator: Badlands,"[Action, Science Fiction, Adventure]","[survival, outcast, alien planet, grand, team ...","[Elle Fanning, Dimitrius Schuster-Koloamatangi...","[John Thomas, Jim Thomas, Patrick Aison, Dan T...","Cast out from his clan, a young Predator finds...",7.292,300
3,1054867,One Battle After Another,"[Action, Thriller, Crime]","[california, based on novel or book, usa–mexic...","[Leonardo DiCaprio, Sean Penn, Chase Infiniti,...","[Adam Somner, Paul Thomas Anderson, Cassandra ...",Washed-up revolutionary Bob exists in a state ...,7.660,1282
4,1197137,Black Phone 2,"[Horror, Thriller]","[mask, child abuse, dreams, sibling relationsh...","[Ethan Hawke, Mason Thames, Madeleine McGraw, ...","[Jason Blum, C. Robert Cargill, Scott Derricks...","Four years after escaping The Grabber, Finney ...",7.131,467


In [33]:
total_end.shape

(5003, 9)

In [27]:
total_end.isna().sum()

movie_id        0
title           0
genre_movies    0
keyword_used    0
cast_names      0
crew_names      0
overview        5
vote_average    0
vote_count      0
dtype: int64

In [36]:
total_end.fillna("", inplace=True)

print(total_end.isna().sum())

movie_id        0
title           0
genre_movies    0
keyword_used    0
cast_names      0
crew_names      0
overview        0
vote_average    0
vote_count      0
dtype: int64


In [54]:
total_end.head()

,movie_id,title,genre_movies,keyword_used,cast_names,crew_names,overview,vote_average,vote_count
0,1062722,Frankenstein,"[drama, horror, fantasy]","[monster, basedonnovelorbook, supernatural, ma...","[oscarisaac, jacobelordi, christophwaltz, miag...","[jmilesdale, keiththompson, dennisberardi, ale...",drvictorfrankensteinabrilliantbutegotisticalsc...,7.873,1250
1,1248226,Playdate,"[action, comedy, family]","[stayathomedad, accountant, actioncomedy]","[kevinjames, alanritchson, sarahchalke, islafi...","[jasonbenoit, jeffreygreenstein, neilgoldman, ...",whenoutofworkaccountantbrianjoinsstayathomedad...,6.600,127
2,1242898,Predator: Badlands,"[action, sciencefiction, adventure]","[survival, outcast, alienplanet, grand, teamup...","[ellefanning, dimitriusschusterkoloamatangi, r...","[johnthomas, jimthomas, patrickaison, dantrach...",castoutfromhisclanayoungpredatorfindsanunlikel...,7.292,300
3,1054867,One Battle After Another,"[action, thriller, crime]","[california, basedonnovelorbook, usa–mexicobor...","[leonardodicaprio, seanpenn, chaseinfiniti, be...","[adamsomner, paulthomasanderson, cassandrakulu...",washeduprevolutionarybobexistsinastateofstoned...,7.660,1282
4,1197137,Black Phone 2,"[horror, thriller]","[mask, childabuse, dreams, siblingrelationship...","[ethanhawke, masonthames, madeleinemcgraw, dem...","[jasonblum, crobertcargill, scottderrickson, s...",fouryearsafterescapingthegrabberfinneyblakeiss...,7.131,467


In [56]:
def clean_text(text):
    # If the value is a list, join it
    if isinstance(text, list):
        text = " ".join(text)

    text = text.lower()                     # lowercase
    text = re.sub(r"[^a-z\s]", " ", text)   # remove punctuation + numbers
    text = re.sub(r"\s+", " ", text).strip() # remove extra spaces

    return text


In [57]:
cols = ["genre_movies", "keyword_used", "cast_names", "crew_names", "overview"]

for c in cols:
    total_end[c] = total_end[c].apply(clean_text)


In [75]:
total_end["combined"] = (
    total_end["genre_movies"] + " " +
    total_end["keyword_used"] + " " +
    total_end["cast_names"] + " " +
    total_end["crew_names"] + " " +
    total_end["overview"]
)

total_end['title'] = total_end['title'].str.lower()

In [76]:
total_end.head()

,movie_id,title,genre_movies,keyword_used,cast_names,crew_names,overview,vote_average,vote_count,combined
0,1062722,frankenstein,drama horror fantasy,monster basedonnovelorbook supernatural madsci...,oscarisaac jacobelordi christophwaltz miagoth ...,jmilesdale keiththompson dennisberardi alexand...,drvictorfrankensteinabrilliantbutegotisticalsc...,7.873,1250,drama horror fantasy monster basedonnovelorboo...
1,1248226,playdate,action comedy family,stayathomedad accountant actioncomedy,kevinjames alanritchson sarahchalke islafisher...,jasonbenoit jeffreygreenstein neilgoldman mark...,whenoutofworkaccountantbrianjoinsstayathomedad...,6.600,127,action comedy family stayathomedad accountant ...
2,1242898,predator: badlands,action sciencefiction adventure,survival outcast alienplanet grand teamup tense,ellefanning dimitriusschusterkoloamatangi rohi...,johnthomas jimthomas patrickaison dantrachtenb...,castoutfromhisclanayoungpredatorfindsanunlikel...,7.292,300,action sciencefiction adventure survival outca...
3,1054867,one battle after another,action thriller crime,california basedonnovelorbook usa mexicoborder...,leonardodicaprio seanpenn chaseinfiniti benici...,adamsomner paulthomasanderson cassandrakulukun...,washeduprevolutionarybobexistsinastateofstoned...,7.660,1282,action thriller crime california basedonnovelo...
4,1197137,black phone 2,horror thriller,mask childabuse dreams siblingrelationship win...,ethanhawke masonthames madeleinemcgraw demi nb...,jasonblum crobertcargill scottderrickson scott...,fouryearsafterescapingthegrabberfinneyblakeiss...,7.131,467,horror thriller mask childabuse dreams sibling...


In [77]:
tdidf = TfidfVectorizer(max_features=50000)
t_matrix = tdidf.fit_transform(total_end['combined'])

In [78]:
cosine_sim = cosine_similarity(t_matrix)

In [79]:
indices = pd.Series(total_end.index, index=total_end['title']).drop_duplicates()

def recommend(movie_title, top_n=10):
    if movie_title not in indices:
        return ["Movie not found!"]
    
    idx = indices[movie_title]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    sim_scores = sim_scores[1: top_n + 1]
    movie_indices = [i[0] for i in sim_scores]
    
    return total_end['title'].iloc[movie_indices].tolist()


In [86]:
to_recommend = input("Enter the Movie to recommend for: ").strip()

recommend(to_recommend)

['the black phone',
 'sinister 2',
 'sinister',
 'urban legends: final cut',
 'the gorge',
 'the exorcism of emily rose',
 'deliver us from evil',
 'the witch',
 'm3gan 2.0',
 'the shining']